In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,547 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,304 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,036 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,306 kB]
Get:13 http://archi

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales_df = spark.read.option('header','true').csv(SparkFiles.get('home_sales_revised.csv'), inferSchema = True, sep=",")

home_sales_df.show(5)


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [4]:
# 2. Create a temporary view of the DataFrame.
from posixpath import sep

home_sales_df.createOrReplaceTempView('home_sales')


In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
select round(avg(price),2) as average_price,
  date_built as year
from home_sales
where
bedrooms = 4
group by date_built

"""

spark.sql(query).show()

+-------------+----+
|average_price|year|
+-------------+----+
|    307908.86|2015|
|    299999.39|2013|
|    299073.89|2014|
|    298233.42|2012|
|    296050.24|2016|
|    296800.75|2010|
|     302141.9|2011|
|    296576.69|2017|
+-------------+----+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query2 = """
select round(avg(price),2) as average_price,
  date_built as year
from home_sales
where
bedrooms = 3
group by date_built

"""

spark.sql(query2).show()


+-------------+----+
|average_price|year|
+-------------+----+
|    299061.15|2015|
|    302956.63|2013|
|    302426.89|2014|
|    298451.99|2012|
|    302937.87|2016|
|    297001.32|2010|
|     300603.7|2011|
|    300313.63|2017|
+-------------+----+



In [8]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query3 = """
select round(avg(price),2) as average_price,
  date_built as year
from home_sales
where
bedrooms = 3
and
bathrooms = 3
and
floors = 2
and
sqft_living >= 2000
group by date_built

"""

spark.sql(query3).show()


+-------------+----+
|average_price|year|
+-------------+----+
|    297609.97|2015|
|    303676.79|2013|
|    298264.72|2014|
|    307539.97|2012|
|     293965.1|2016|
|    285010.22|2010|
|    276553.81|2011|
|    280317.58|2017|
+-------------+----+



In [23]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query4 = """
select
  round(avg(price),2) as average_price,
  view
from
    home_sales
where
price >= 350000
group by
  view
order by
  view

"""

spark.sql(query4).show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|average_price|view|
+-------------+----+
|    403848.51|   0|
|    401044.25|   1|
|    397389.25|   2|
|     398867.6|   3|
|    399631.89|   4|
|    401471.82|   5|
|    395655.38|   6|
|    403005.77|   7|
|    398592.71|   8|
|    401393.34|   9|
|    401868.43|  10|
|    399548.12|  11|
|    401501.32|  12|
|    398917.98|  13|
|    398570.03|  14|
|     404673.3|  15|
|    399586.53|  16|
|    398474.49|  17|
|    399332.91|  18|
|    398953.17|  19|
+-------------+----+
only showing top 20 rows

--- 0.7631552219390869 seconds ---


In [13]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales ")

DataFrame[]

In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [15]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query5 = """
select round(avg(price),2) as average_price,
  view
from home_sales
where
price >= 350000
group by view
order by view

"""

spark.sql(query5).show()



print("--- %s seconds ---" % (time.time() - start_time))


+-------------+----+
|average_price|view|
+-------------+----+
|    403848.51|   0|
|    401044.25|   1|
|    397389.25|   2|
|     398867.6|   3|
|    399631.89|   4|
|    401471.82|   5|
|    395655.38|   6|
|    403005.77|   7|
|    398592.71|   8|
|    401393.34|   9|
|    401868.43|  10|
|    399548.12|  11|
|    401501.32|  12|
|    398917.98|  13|
|    398570.03|  14|
|     404673.3|  15|
|    399586.53|  16|
|    398474.49|  17|
|    399332.91|  18|
|    398953.17|  19|
+-------------+----+
only showing top 20 rows

--- 0.696181058883667 seconds ---


In [16]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_parquet_partitioned")

In [17]:
# 11. Read the parquet formatted data.
home_sales_parquet_partitioned = spark.read.parquet("home_sales_parquet_partitioned")

In [18]:
# 12. Create a temporary table for the parquet data.
home_sales_parquet_partitioned.createOrReplaceTempView("home_sales_parquet_partitioned_view")

In [19]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query6 = """
select
  round(avg(price),2) as average_price,
  view
from
  home_sales_parquet_partitioned_view
where
  price > 350000
group by
  view
order by
  view

"""

spark.sql(query6).show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|average_price|view|
+-------------+----+
|    403848.51|   0|
|    401044.25|   1|
|    397389.25|   2|
|     398867.6|   3|
|    399631.89|   4|
|    401471.82|   5|
|    395655.38|   6|
|    403005.77|   7|
|    398592.71|   8|
|    401393.34|   9|
|    401868.43|  10|
|    399548.12|  11|
|    401501.32|  12|
|    398917.98|  13|
|    398570.03|  14|
|     404673.3|  15|
|    399586.53|  16|
|    398474.49|  17|
|    399332.91|  18|
|    398953.17|  19|
+-------------+----+
only showing top 20 rows

--- 1.0736372470855713 seconds ---


In [20]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [21]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales")

False

- The runtime for the original data was 0.7859 seconds.
- The runtime for the cached data was 0.6962 seconds.
- The runtime for the parquet and partitioned data was 1.074 seconds.

